In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#!unzip -u /content/drive/MyDrive/pixels.zip -d /tmp/images/

Archive:  /content/drive/MyDrive/pixels.zip
warning [/content/drive/MyDrive/pixels.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [/content/drive/MyDrive/pixels.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #109299).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [15]:
from PIL import Image
import torch
import pandas as pd
import os, sys
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, transforms, models
import torch.nn as nn

In [4]:
class AnomalyDataset(Dataset):
    def __init__(self,csvFile,rootDir,transform=None):
        self.annotations = pd.read_csv(csvFile)
        self.rootDir = rootDir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
        

    def __getitem__(self, index):
        imgPath = os.path.join(self.rootDir,self.annotations.iloc[index,0])
        image = io.imread(imgPath)
        yLabel = torch.tensor(int(self.annotations.iloc[index,1]))
        if self.transform:
            image = self.transform(Image.fromarray(image))
        
        return (image,yLabel)    

In [ ]:
statisticsDir = "/home/xsafon00/pzidovsky/statistics/"
modelsDir = "/home/xsafon00/pzidovsky/models/"
imagesDir = "/home/xsafon00/pzidovsky/data/"

os.mkdir(modelsDir)
os.mkdir(statisticsDir)

In [5]:
inputSize = 224
batchSize = 40
learninigRate = 0.005
epochs = 50

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

In [ ]:
dataset = AnomalyDataset(csvFile=os.path.join(imagesDir,"LabelsCat.csv"), rootDir=imagesDir,transform=transform)

trainDatasetLength = int(len(dataset)*0.9)

trainSet, testSet = torch.utils.data.random_split(dataset,[trainDatasetLength,len(dataset)-trainDatasetLength])
trainLoader = torch.utils.data.DataLoader(trainSet,batch_size=batchSize,shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet,batch_size=batchSize,shuffle=True)

In [ ]:
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b4', pretrained=True)
for param in model.parameters():
    param.requires_grad_(False)
blocks_to_retrain = 1
count = 0
for name, block in model.named_children():
    if count >= (4 - blocks_to_retrain):
        print(f'Unfreeze block {name}')
        for pname, params in block.named_parameters():
            if 'bn' not in pname:
                params.requires_grad = True
    else:
        print(f'Keep block {name} frozen')
    count += 1
model.classifier[3] = nn.Linear(in_features=1792, out_features=20)
model.eval()

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.CrossEntropyLoss()
#model = model.cuda()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learninigRate)

In [ ]:
trainLoss = []
testLoss = []
trainAcc = []
testAcc = []

for epoch in range(1,epochs+1):
    print(epoch)
    runningLoss = .0
    correct = 0
    total = 0
    for i, (inputs,labels) in enumerate(trainLoader):
        #inputs = inputs.cuda()
        #labels = labels.cuda()
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        inputs.size()
        outputs = model(inputs)
        loss = criterion(outputs, labels) 

        loss.backward()

        optimizer.step()

        runningLoss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i%10 == 0:
            print(i)

    runningLoss /= len(trainLoader)
    trainLoss.append(runningLoss)
    runningAcc = correct / total
    trainAcc.append(runningAcc)

    file_object = open(statisticsDir+'learningOutput.txt', 'a')
    file_object.write("\nEpoch: {}\t".format(epoch))
    file_object.write('\tTrain Acc. => {:.3f}%\t'.format(100*runningAcc))
    file_object.write('\tTrain Loss => {:.5f}'.format(runningLoss))
    file_object.close()
    if (epoch-1) % 2 == 0:
        path = os.path.join(modelsDir, 'efficientnet_b4_'+str(epoch)+'.pth')
        torch.save(model.cpu().state_dict(), path)
        #model.cuda()
        model.to(device)

    with torch.no_grad():
        trueLabels = []
        predictedLabels = []
        correct = 0
        total = 0
        testRunningLoss = 0
        for data in testLoader:
            inputs, labels = data
            #inputs = inputs.cuda()
            #labels = labels.cuda()
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            testRunningLoss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for i in range(len(predicted)):
                trueLabels.append(labels[i].item())
                predictedLabels.append(predicted[i].item())
        
        testRunningLoss /= len(testLoader)
        testLoss.append(testRunningLoss)
        testRunningAcc = correct /total
        testAcc.append(testRunningAcc)

        if (epoch-1) % 1 == 0:
            file_object = open(statisticsDir+'learningOutput.txt', 'a')
            file_object.write("\nEpoch: {}\t".format(epoch))
            file_object.write('\tTest Acc. => {:.3f}%\t'.format(100 * testRunningAcc))
            file_object.write('\tTest Loss => {:.5f}'.format(testRunningLoss))
            file_object.close()

            predict_file = open(os.path.join(statisticsDir,"predict"+str(epoch)+".txt"),'a')
            predict_file.write(str(trueLabels))
            predict_file.write(';')
            predict_file.write(str(predictedLabels))
            predict_file.close()


1
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210

In [ ]:
path = os.path.join(modelsDir, 'efficientnet_b4_complete.pth')
torch.save(model,path)